In [7]:
import urllib.request
import json
import os
import ssl
from azureml.contrib.services.aml_request import AMLRequest, rawhttp
from azureml.contrib.services.aml_response import AMLResponse
from PIL import Image
import base64

def allowSelfSignedHttps(allowed):
    # bypass the server certificate verification on client side
    if allowed and not os.environ.get('PYTHONHTTPSVERIFY', '') and getattr(ssl, '_create_unverified_context', None):
        ssl._create_default_https_context = ssl._create_unverified_context

allowSelfSignedHttps(True) # this line is needed if you use self-signed certificate in your scoring service.

# Request data goes here
# The example below assumes JSON formatting which may be updated
# depending on the format your endpoint expects.
# More information can be found here:
# https://docs.microsoft.com/azure/machine-learning/how-to-deploy-advanced-entry-script
image_path = 'C:\\Users\\nehab\\OneDrive\\Desktop\\AIPI_561_Individual_Project_2\\brain tumor\\yes\\Y1.jpg'
with open(image_path, 'rb') as image_file:
    image_data = base64.b64encode(image_file.read()).decode('utf-8')
data =  {
  "Inputs": {
    "WebServiceInput0": [
      {
        "image": f"data:image/jpeg;base64,{image_data}",
        "id": 1,
        "category": "yes"
      }
    ]
  },
  "GlobalParameters": {}
}

body = str.encode(json.dumps(data))

url = 'http://73bfb30b-5082-4c7f-bf50-c9622e24a578.eastus.azurecontainer.io/score'
# Replace this with the primary/secondary key or AMLToken for the endpoint
api_key = 'bd9ubgd1pTJYl5WaFT2SanRQu3KDvdiK'
if not api_key:
    raise Exception("A key should be provided to invoke the endpoint")


headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ api_key)}

req = urllib.request.Request(url, body, headers)

try:
    response = urllib.request.urlopen(req)

    result = response.read()
    print(result)
except urllib.error.HTTPError as error:
    print("The request failed with status code: " + str(error.code))

    # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
    print(error.info())
    print(error.read().decode("utf8", 'ignore'))

b'{"Results": {"WebServiceOutput0": [{"category": "yes", "id": 1, "Scored Probabilities_no": 7.675535925955046e-06, "Scored Probabilities_yes": 0.9999923706054688, "Scored Labels": "yes"}]}}'
